# Agents as Tools with Strands Agents (亚马逊云科技中国区)


"Agents as Tools"（工具型代理）是 AI 系统中的一种多Agnet架构模式，它将专门的 AI agents 包装成可调用的函数（工具），供其他 agents 使用。这创建了一个层次结构：

1. 一个主要的"orchestrator"（编排器）agent 处理用户交互并确定调用哪个专门的 agent

2. 专门的"tool agents"在被 orchestrator 调用时执行特定领域的任务

这种方法模仿了人类团队动态，其中管理者协调专家，每个专家都带来独特的专业知识来解决复杂问题。与单个 agent 尝试处理所有事情不同，任务被委派给最合适的专业 agent。



## 主要优势和核心原则

"Agents as Tools"模式提供了几个优势：

- 关注点分离：每个 agent 都有一个专注的责任领域，使系统更容易理解和维护
- 层次化委派：orchestrator 决定调用哪个专家，创建了清晰的命令链
- 模块化架构：专家可以独立添加、移除或修改，而不影响整个系统
- 提升性能：每个 agent 可以拥有为其特定任务优化的定制系统提示和工具

In [1]:
#!pip install -r requirements.txt
!pip install strands-agents
!pip install strands-agents-tools
!pip install strands-agents[openai]


In [2]:
import os

from strands import Agent, tool
from strands_tools import file_write
from strands.models.openai import OpenAIModel

在这个模块中，我们将创建一个基于 orchestrator 的多代理工作流程。


<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

我们还将探索 `use_llm`，它允许我们创建嵌套的 agents。

## Research Agent

让我们首先创建一个带有 http_request 工具的基础研究助手。

In [3]:
RESEARCH_ASSISTANT_PROMPT = """你是一个专业的研究助手。只专注于针对研究问题提供基于事实、有充分来源的信息。在可能的情况下始终引用你的信息来源。"""

In [4]:
model = OpenAIModel(
    client_args={
        "api_key": ##"此处填写硅基流动key",
         "base_url": "https://api.siliconflow.cn"
    },
    # **model_config
    model_id="deepseek-ai/DeepSeek-V3",
    params={
        "max_tokens": 10000,
        "temperature": 0.7,
    }
)


In [5]:
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    model=model
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "亚马逊云科技的概述"
# Call the agent and return its response
response = research_agent(query)

亚马逊云科技（Amazon Web Services, AWS）是亚马逊公司（Amazon.com）旗下的云计算服务平台，于2006年正式推出。作为全球领先的云服务提供商，AWS提供包括计算、存储、数据库、网络、人工智能、机器学习、数据分析、安全等在内的200多种全功能云服务，服务范围覆盖全球多个区域和可用区。

### **核心服务分类**
1. **计算服务**  
   - **Amazon EC2**（弹性计算云）：提供可扩展的虚拟服务器。  
   - **AWS Lambda**：无服务器计算服务，支持事件驱动的代码执行。  

2. **存储服务**  
   - **Amazon S3**（简单存储服务）：对象存储服务，适用于大数据和备份。  
   - **Amazon EBS**（弹性块存储）：为EC2提供块级存储。  

3. **数据库服务**  
   - **Amazon RDS**：托管的关系型数据库服务（支持MySQL、PostgreSQL等）。  
   - **Amazon DynamoDB**：全托管的NoSQL数据库服务。  

4. **网络服务**  
   - **Amazon VPC**（虚拟私有云）：用于隔离云资源。  
   - **AWS Direct Connect**：提供专线连接企业数据中心和AWS。  

5. **人工智能与机器学习**  
   - **Amazon SageMaker**：用于构建、训练和部署机器学习模型。  
   - **Amazon Rekognition**：图像和视频分析服务。  

6. **安全与合规**  
   - **AWS IAM**（身份和访问管理）：管理用户权限。  
   - **AWS Shield**：提供DDoS防护。  

7. **数据分析**  
   - **Amazon Redshift**：数据仓库解决方案。  
   - **AWS Glue**：ETL（提取、转换、加载）服务。  

### **市场地位**
根据**Synergy Research Group**（2023年数据），AWS在全球云计算市场占据约**34%**的份额，领先于微软Azure（约22%）和谷歌云（约11%）[1]。  

### **全球基础设施**
AW

现在我们可以将这个 agent 包装为一个工具，允许其他 agents 与之交互。

#### 使用 Agent as Tools 的最佳实践

实现"Agents as Tools"模式时,请注意：

1. 清晰的工具文档：编写描述性的文档字符串，解释 agent 的专业知识
2. 聚焦的系统提示：保持每个专业 agent 严格关注其领域
3. 适当的响应处理：使用一致的模式提取和格式化响应
4. 工具选择指导：为 orchestrator 提供明确的标准，说明何时使用每个专业 agent

In [6]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
            model=model
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

现在让我们遵循最佳实践创建 `product_recommendation_assistant`、`trip_planning_assistant` 和 `orchestrator` agent。

### 产品推荐助手

In [7]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            system_prompt="""你是一个专业的产品推荐助手。
根据用户偏好提供个性化产品建议。始终引用你的信息来源。""",
            model=model
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [8]:
product_recommendation_assistant("关于电动汽车的产品推荐")

以下是一些值得考虑的电动汽车推荐，基于2023年市场数据（来源：CleanTechnica、Consumer Reports、EV Database）：

### 1. **高端性能车型**
- **Tesla Model S Plaid**  
  - 续航：637公里（EPA）  
  - 0-100km/h：2.1秒  
  - 亮点：三电机、17英寸触控屏、尖端自动驾驶硬件  
  - 参考价：约$110,000  

- **Lucid Air Grand Touring**  
  - 续航：830公里（EPA）  
  - 0-100km/h：3秒  
  - 亮点：豪华内饰、超快充电（300kW）  
  - 参考价：约$138,000  

### 2. **家用主流车型**
- **Tesla Model Y**（2023年全球最畅销电动车）  
  - 续航：530公里（长续航版）  
  - 亮点：超大储物空间、OTA更新、超级充电网络  
  - 参考价：约$50,000  

- **Ford Mustang Mach-E**  
  - 续航：490公里（加州路由版）  
  - 亮点：运动调校、Sync 4A系统、性价比高  
  - 参考价：约$47,000  

### 3. **高性价比车型**
- **Chevrolet Bolt EV/EUV**（2023年停产，但仍有库存优惠）  
  - 续航：416公里  
  - 亮点：最便宜的电动车之一，支持DC快充  
  - 参考价：约$26,000（含补贴后）  

- **Volkswagen ID.4**  
  - 续航：450公里（Pro版）  
  - 亮点：欧洲风格设计、免费充电（Electrify America网络）  
  - 参考价：约$38,000  

### 4. **中国品牌（全球市场）**
- **BYD Atto 3**（海外版元PLUS）  
  - 续航：420公里（WLTP）  
  - 亮点：刀片电池、旋转中控屏  
  - 参考价：约$35,000（亚太/欧洲）  

- **MG ZS EV**  
  - 续航：440公里（长续航版）  
  - 亮点：性价比高，支持V2L（车辆对外放电）  
  - 参考价：约$30,000  

'以下是一些值得考虑的电动汽车推荐，基于2023年市场数据（来源：CleanTechnica、Consumer Reports、EV Database）：\n\n### 1. **高端性能车型**\n- **Tesla Model S Plaid**  \n  - 续航：637公里（EPA）  \n  - 0-100km/h：2.1秒  \n  - 亮点：三电机、17英寸触控屏、尖端自动驾驶硬件  \n  - 参考价：约$110,000  \n\n- **Lucid Air Grand Touring**  \n  - 续航：830公里（EPA）  \n  - 0-100km/h：3秒  \n  - 亮点：豪华内饰、超快充电（300kW）  \n  - 参考价：约$138,000  \n\n### 2. **家用主流车型**\n- **Tesla Model Y**（2023年全球最畅销电动车）  \n  - 续航：530公里（长续航版）  \n  - 亮点：超大储物空间、OTA更新、超级充电网络  \n  - 参考价：约$50,000  \n\n- **Ford Mustang Mach-E**  \n  - 续航：490公里（加州路由版）  \n  - 亮点：运动调校、Sync 4A系统、性价比高  \n  - 参考价：约$47,000  \n\n### 3. **高性价比车型**\n- **Chevrolet Bolt EV/EUV**（2023年停产，但仍有库存优惠）  \n  - 续航：416公里  \n  - 亮点：最便宜的电动车之一，支持DC快充  \n  - 参考价：约$26,000（含补贴后）  \n\n- **Volkswagen ID.4**  \n  - 续航：450公里（Pro版）  \n  - 亮点：欧洲风格设计、免费充电（Electrify America网络）  \n  - 参考价：约$38,000  \n\n### 4. **中国品牌（全球市场）**\n- **BYD Atto 3**（海外版元PLUS）  \n  - 续航：420公里（WLTP）  \n  - 亮点：刀片电池、旋转中控屏  \n  - 参考价：约$35,000（亚太/欧洲）  \n\n- **MG ZS EV**  \n  - 续航：440公里（长续航版）  \n

### 旅行规划助手

In [9]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            system_prompt="""你是一个专业的旅行规划助手。根据用户偏好创建详细的旅行行程。""",
            model=model,
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Orchestrator（编排器） Agent

In [10]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
你是一个将查询路由到专门 agents 的助手：
- 对于研究问题和事实信息 → 使用 research_assistant 工具
- 对于产品推荐和购物建议 → 使用 product_recommendation_assistant 工具
- 对于旅行规划和行程 → 使用 trip_planning_assistant 工具
- 对于不需要专业知识的简单问题 → 直接回答

始终根据用户的查询选择最合适的工具。
"""

In [11]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    model=model,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [12]:
# Example: E-commerce Customer Service System
customer_query = (
    "我在寻找徒步靴。将最终响应写入当前目录。"
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)


Tool #1: product_recommendation_assistant
根据您的需求，这里为您推荐几款适合徒步的优质靴子，涵盖不同场景和预算，并附上来源信息：

### 一、专业级重装徒步靴（适合崎岖地形/负重）
1. **Scarpa Zodiac Plus GTX**  
   - 亮点：Vibram大底+全皮面+Gore-Tex防水，适合技术性山地徒步  
   - 来源：《Outdoor Gearlab》2023年最佳登山靴评测  
   - 价格区间：¥2000-2500

2. **Lowa Renegade GTX Mid**  
   - 亮点：德国工艺+耐磨PU中底，连续5年获《Backpacker》杂志编辑选择奖  
   - 来源：Backpacker 2022年装备指南  

### 二、中量级多功能靴（3季适用）
1. **Salomon Quest 4 GTX**  
   - 亮点：智能鞋带系统+Contagrip高抓地底，被《Gear Patrol》评为最佳全能靴  
   - 实测数据：比前代减重15%（来源：Salomon实验室报告）  

2. **Merrell Moab 3 Mid**  
   - 亮点：性价比之王，Air Cushion缓震技术  
   - 来源：美国REI销量冠军（2023年Q2数据）  

### 三、轻量快速徒步鞋（适合轻装/成熟步道）
1. **Altra Lone Peak ALL-WTHR**  
   - 特色：宽楦设计+零落差中底，获《Trail Runner》2023年最佳徒步跑鞋  
   - 适合：追求自然步态的用户  

2. **HOKA Anacapa Mid GTX**  
   - 亮点：超厚缓震中底+环保材料（50%再生材料）  
   - 来源：HOKA 2023可持续发展报告  

### 选购建议：
1. **试穿要点**：建议下午脚部肿胀时试穿，预留1cm脚尖空间（来源：美国足病医学会建议）  
2. **防水选择**：Gore-Tex适合多雨环境，但透气性较eVent略差（来源：《Outside》材质对比测试）  
3. **保养提示**：皮质靴需定期用专用护理油（如Sno-Seal），可延长2-3倍使用寿命（来源：Scarpa保养手册）  



╔══════════ File Write Operation ══════════╗
║                                          ║
║ Path: ./hiking_boots_recommendations.txt ║
║ Size: 1087 characters                    ║
║                                          ║
╚══════════════════════════════════════════╝

╔═══════════════════════ Write Successful ════════════════════════╗
║ File written successfully to ./hiking_boots_recommendations.txt ║
╚═════════════════════════════════════════════════════════════════╝

徒步靴的推荐已成功写入当前目录下的 `hiking_boots_recommendations.txt` 文件中。您可以在文件中查看详细的推荐内容和选购建议。如果需要进一步的帮助或其他特定需求，请随时告诉我！

让我们查看 orchestrator 的消息。在这里你可以看到该 agent 决定使用子 agent 作为工具

In [ ]:
orchestrator.messages

In [ ]:
customer_query = "你能帮我规划我去成都的旅行吗？"

response = orchestrator(customer_query)

In [ ]:
orchestrator.messages

### 调用多个 agents

In [17]:
orchestrator.messages = []

In [ ]:
query = "你能对中国进行研究吗？同时帮我规划一个7天的旅行."

orchestrator(query)

在幕后，orchestrator 将：
1. 首先调用 research_assistant
2. 然后调用 trip_planning_assistant
3. 将这些专业响应组合成一个完整的答案，以解决这两个查询

### 顺序 Agent 通信模式

agent 工具还可以将多个 agents 组合在一起。在这个例子中，我们将 `research_agent` 的输出提供给 `summary_agent`，并返回汇总的响应。

In [15]:
# 定义用户查询
topic = "generative Ai"
# 创建一个研究agent
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    model=model
)
# 创建一个总结agent
summary_agent = Agent(
    system_prompt="""
    你是一位专注于将复杂信息提炼成清晰、简洁摘要的总结专家。
    你的主要目标是提取详细信息中的关键点、主要论点和关键数据。
    你应该在保持原始内容准确性的同时，使其更易于理解。
    专注于清晰度、简洁性，并突出信息中最重要的方面。
    """,
    model=model
)

print("多个 agents 创建成功！")
print(f"\n🔍 研究 AGENT 正在处理: {topic}\n") 
try:
    # Agent 1: 调用研究agent
    research_response = research_agent(
        f"请收集关于{topic}的综合信息。"
    )
    print(research_response)
    print(research_response.message)
    print("---------------------")
    research_text = research_response.message['content'][0]["text"]
    print("\n✂️ 总结 AGENT 正在提炼研究内容\n")
    
    # Agent 2: 请总结agent创建简洁摘要
    summary_response = summary_agent(
        f"请为这项研究创建一个简洁的摘要: {research_text}"
    )
    summary_text = summary_response.message['content'][0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"研究助手出错: {str(e)}")

多个 agents 创建成功！

🔍 研究 AGENT 正在处理: generative Ai

以下是关于生成式人工智能（Generative AI）的综合信息，涵盖定义、核心技术、应用领域、挑战以及未来趋势，均基于权威来源和研究报告：

---

### **1. 定义与核心概念**
生成式AI（Generative AI）指能够自主生成新内容（如文本、图像、音频、代码等）的人工智能系统。其核心是通过学习大量数据分布，模拟并创造类似但全新的输出。  
- **与判别式AI的区别**：判别式模型（如分类器）用于区分数据类别，而生成式模型专注于创造数据（OpenAI, 2021）。

---

### **2. 关键技术**
#### **(1) 生成对抗网络（GANs）**  
- 由生成器（Generator）和判别器（Discriminator）组成，通过对抗训练生成逼真数据（Goodfellow et al., 2014）。  
- **应用**：图像生成（如Deepfake）、艺术创作（如Artbreeder）。

#### **(2) 变分自编码器（VAEs）**  
- 通过编码-解码结构学习数据潜在分布，适合生成连续数据（如分子设计）（Kingma & Welling, 2013）。

#### **(3) 自回归模型（如GPT系列）**  
- 基于Transformer架构，通过预测序列中下一个token生成文本、代码等（OpenAI, 2020）。  
- **示例**：ChatGPT（文本）、Codex（代码）。

#### **(4) 扩散模型（Diffusion Models）**  
- 通过逐步去噪过程生成高质量图像，如Stable Diffusion和DALL·E 2（Ho et al., 2020）。

---

### **3. 主要应用领域**
#### **(1) 内容创作**  
- **文本**：自动写作、对话系统（如ChatGPT）。  
- **图像/视频**：AI绘画（MidJourney）、视频合成（Runway ML）。  
- **音乐**：AIVA生成原创曲目。

#### **(2) 医疗与科研**  
- 药物分子设计（如Atomwise）、医学影像合成（ArXiv, 2022）。

#### **(3

## 恭喜！

你已经学会了如何在 Strands Agents 中将 agents 用作工具，以创建更复杂的代理应用程序